In [62]:
import requests
import re
import pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [2]:
# Get base page
base_url = "http://www.mediotiempo.com/liga/futbol/ligamx/tabla-general/"
base_page = requests.get(base_url).text
base_soup = BeautifulSoup(base_page, "lxml")

In [3]:
tournament_container = base_soup.find("div", { "class" : "dropdown-container" })
ul = tournament_container.find('ul')
tournaments = []
for li in ul.findAll('li'):
    tournaments.append(li.get('value'))

In [4]:
# Get tables
c = { 'Team': 0, 'PTS':1, 'JJ':2, 'DG':3, 'JG':4,'JE':5, 'JP':6, 'GF': 7, 'GC': 8 }
print("Tournaments", len(tournaments))
scraped = {}
for tournament in tournaments:
    results = []
    url = urljoin(base_url, tournament)
    tournament_page = requests.get(url).text
    tournament_soup = BeautifulSoup(tournament_page, "lxml")
    tables = tournament_soup.findAll("div", { "class" :'table-positions' })
    for table in tables:
        # need to find the table inside div.scroll:
        table = table.find('div', {'class':'scroll'}).find('table', {'class':'mt-table'})
        rows = table.tbody.findAll('tr')
        for row in rows:
            tds = row.findAll('td')
            team = tds[c['Team']].text.strip()
            pts =  tds[c['PTS']].text.strip()
            jj =  tds[c['JJ']].text.strip()
            dg =  tds[c['DG']].text.strip()
            jg =  tds[c['JG']].text.strip()
            je =  tds[c['JE']].text.strip()
            jp =  tds[c['JP']].text.strip()
            gf =  tds[c['GF']].text.strip()
            gc =  tds[c['GC']].text.strip()
            team_stat = {
                'team':team,
                'pts':pts,
                'jj':jj,
                'dg':dg,
                'jg':jg,
                'je':je,
                'jp':jp,
                'gf':gf,
                'gc':gc
            }
            results.append(team_stat)
    scraped[tournament] = results

Tournaments 69


In [46]:
torneo_largo = re.compile('(\w+)-([0-9]{4})-+([0-9]{4})')
torneo_corto = re.compile('(\w+)-([0-9]{4})')

In [74]:
dos = set(['invierno', 'apertura'])
uno = set(['verano', 'clausura', 'bicentenario'])
intermediate = []
for torneo in scraped:
    match_torneo_largo = torneo_largo.search(torneo)
    match_torneo_corto = torneo_corto.search(torneo)
    name = ''
    if match_torneo_largo:
        t = match_torneo_largo.group(1)
        inicio = int(match_torneo_largo.group(2).upper())
        fin = int(match_torneo_largo.group(3).upper())
        if t == "temporada":
            name = "Temporada de " + str(inicio) + " a "+ str(fin)
        if t == "liguilla":
            name = "Liguilla de " + str(inicio) + " a "+ str(fin)
    elif match_torneo_corto:
        t = match_torneo_corto.group(1)
        c = int(match_torneo_corto.group(2).upper())
        if t in dos:
            name = "Torneo corto " + str(c)  + "-2"
        if t in uno:
            name = "Torneo corto " + str(c)  + "-1"
    for result in scraped[torneo]:
        intermediate.append([
            torneo,
            result['team'],
            result['pts'],
            result['jj'],
            result['dg'],
            result['jg'],
            result['je'],
            result['jp'],
            result['gf'],
            result['gc']
        ])
tournament_df = pd.DataFrame(intermediate)
tournament_df.columns = ['id', 'team', 'pts', 'jj', 'dg', 'jg','je', 'jp', 'gf', 'gc']
tournament_df.set_index('id', inplace=True)
print(tournament_df.tail())

                        team pts  jj  dg  jg  je  jp  gf  gc
id                                                          
temporada-1975-1976    Pumas  61  38  21  15  16   7  56  35
temporada-1975-1976     León  61  38  17  15  16   7  49  32
temporada-1975-1976   Toluca  54  38   8  13  15  10  44  36
temporada-1975-1976   Laguna  51  38   7  11  18   9  44  37
temporada-1975-1976  Jalisco  44  38  -1  10  14  14  58  59


In [73]:
tournament_df.to_csv('mt-scraper/tournaments.csv')